In [1]:
from typing import *

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from datetime import datetime
import os


In [2]:
FILES = sorted([os.path.join("Data/Minutes", path) for path in os.listdir("Data/Minutes")[:5]])
DATA_PATH = ("temp/train", "temp/test")
SEQ_LEN = 64


In [3]:
class DataSetTooSmall(Exception):
	pass


In [9]:
class DataPreparer:
	def __init__(
			self, 
			seq_len, 
			files: List[str], 
			train_output_path: str,
			test_output_path: str,
			test_split_ratio: float = 0.1,
			x_column_headers: List[str] = None, 
			y_column_header: str = "y",
			max_rows: int = 1000000
	):
		self.__seq_len = seq_len
		self.__files = files
		self.__train_output_path, self.__test_output_path = train_output_path, test_output_path
		self.__test_split_ratio = test_split_ratio
		self.__max_rows = max_rows
		self.__x_column_headers = x_column_headers
		if x_column_headers is None:
			self.__x_column_headers = [str(i) for i in range(seq_len)]
		self.__y_column_header = y_column_header
		
	def __get_currency_pairs(self, df: pd.DataFrame) -> List[Tuple[str, str]]:
		DELIMITER = "/"
		return [
			(pair.split(DELIMITER)[0], pair.split(DELIMITER)[1]) for pair in
			set(df["base_currency"] + DELIMITER + df["quote_currency"])
		]  # TODO FIND A CLEAR WAY

	def __prepare_for_pair(self, sequence: np.ndarray, seq_len: int):
		data_len = sequence.shape[0] - seq_len
		X = np.zeros((data_len, seq_len))
		y = np.zeros((data_len,))
		for i in range(data_len):
			X[i] = sequence[i:i + seq_len]
			if sequence[i] > sequence[i - 1]:
				y[i] = 1
			else:
				y[i] = 0
		return X, y

	def __prepare_data(self, data: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
		currency_pairs = self.__get_currency_pairs(data)

		X = []
		y = []

		for base_currency, quote_currency in currency_pairs:
			pair_sequence = data[data["base_currency"] == base_currency][
				data[data["base_currency"] == base_currency]["quote_currency"] == quote_currency
			]["c"].to_numpy()  # TODO
			pair_X, pair_y = self.__prepare_for_pair(pair_sequence, self.__seq_len)
			X += list(pair_X)
			y += list(pair_y)

		return np.array(X), np.array(y)

	def __split_data(self, X, y) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
		return train_test_split(X, y, test_size=self.__test_split_ratio, random_state=42)

	def __load_file(self, file_name: str) -> pd.DataFrame:
		return pd.read_csv(file_name, index_col=0)

	def __process_file(self, file: str) -> Tuple[pd.DataFrame, pd.DataFrame]:

		raw_data = self.__load_file(file)
		if len(raw_data) < self.__seq_len:
			raise DataSetTooSmall()
		X, y = self.__prepare_data(raw_data)
		X_train, X_test, y_train, y_test = self.__split_data(X, y)
		train_df, test_df = pd.DataFrame(X_train, columns=self.__x_column_headers), pd.DataFrame(X_test, columns=self.__x_column_headers)
		train_df.columns = test_df.columns = self.__x_column_headers
		train_df[self.__y_column_header], test_df[self.__y_column_header] = pd.DataFrame(y_train), pd.DataFrame(y_test)
		return train_df, test_df

	def __create_dfs(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
		return tuple([pd.DataFrame(columns=self.__x_column_headers+[self.__y_column_header]) for i in range(2)])

	def __generate_filename(self) -> str:
		return f"{datetime.now().timestamp()}.csv"
	
	def __save_df(self, df: pd.DataFrame, folder):
		df.to_csv(os.path.join(folder, self.__generate_filename()), index=False)
	
	def __append_df(self, df: pd.DataFrame, new_data: pd.DataFrame, path: str) -> pd.DataFrame:
		if len(df) + len(new_data) <= self.__max_rows:
			return df.append(new_data)
		bound: int = self.__max_rows - len(df)
		old_df = self.__append_df(df, new_data.iloc[:bound], path)
		self.__save_df(old_df, path)
		return new_data[bound:]

	def __append_dfs(self, train_dfs: Tuple[pd.DataFrame, pd.DataFrame], test_dfs: Tuple[pd.DataFrame, pd.DataFrame]) -> Tuple[pd.DataFrame, pd.DataFrame]:
		train_df = self.__append_df(train_dfs[0], train_dfs[1], self.__train_output_path)
		test_df = self.__append_df(test_dfs[0], test_dfs[1], self.__test_output_path)
		return train_df, test_df
		
	def start(self):
		train_df, test_df = self.__create_dfs()
		for i, file in enumerate(self.__files):
			try:
				new_train_data, new_test_data = self.__process_file(file)
			except DataSetTooSmall:
				continue
			train_df, test_df = self.__append_dfs(
				(train_df, new_train_data),
				(test_df, new_test_data)
			)
			print(f"Done: {100*(i+1)/len(self.__files) :.2f}%\tFinished: {file}", end="\r")
		self.__save_df(train_df, self.__train_output_path)
		self.__save_df(test_df, self.__test_output_path)


In [10]:
processor = DataPreparer(
	SEQ_LEN,
	FILES,
	DATA_PATH[0],
	DATA_PATH[1]
)
processor.start()
